In [1]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
from cellpose.io import imread
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def get_measurements(mask,img,props):
    df = sk.measure.regionprops_table(mask,img,properties=props)
    return df

In [ ]:
def scaled_vol(df,voxel):
    scaled_vol = []
    for a in np.asarray(df['area']).astype(int):
        scaled_vol.append(a*voxel)
    vol_df = pd.Series(scaled_vol,name='Volume (um^3)')
    scaled_df = pd.concat([df,vol_df],axis=1)
    return scaled_df

In [ ]:
def get_3D_surfacearea(mask_img,scaled_df):
    array = []
    for i in np.asarray(scaled_df['label']).astype(int):
        obj = mask_img == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0)
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        array.append(surf_area)
    surface_areas = pd.Series(array,name='Surface_Area (um^2)')
    merged_df = pd.concat([scaled_df,surface_areas], axis=1)
    return merged_df

In [ ]:
def save(save_path, img_name, masks, flows, merged_df):
    # make directories if they do not exist
    try:
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        flows_path = os.mkdir(os.path.join(save_path,'flows'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        flows_path = os.path.join(save_path,'flows')
        dataframe_path = os.path.join(save_path,'measurements')
    imsave(os.path.join(masks_path,'masks_'+img_name[:-4]+'.tif'),mask_img)
    imsave(os.path.join(selected_spots_path,'flows_'+img_name[:-4]+'.tif'),selected_spots)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

In [2]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [3]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [4]:
all_files = sorted(glob('E:/ParadaKusz_Lab/Test_Outputs/background_subtracted_imgs/*.tif'))
all_img = list(map(sk.io.imread,all_files))

In [6]:
GMO_2ng_imgs[1].shape

(44, 2048, 2048)

In [15]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, flows, styles, diams = model.eval(all_img[1], diameter=70, channels=channels, z_axis=0,stitch_threshold=0.3,flow_threshold=0.4,cellprob_threshold=-2.0)

creating new log file
2025-05-14 11:08:42,633 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-05-14 11:08:42,634 [INFO] 
cellpose version: 	3.1.1.1 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.6.0+cu118
2025-05-14 11:08:42,637 [INFO] ** TORCH CUDA version installed and working. **
2025-05-14 11:08:42,638 [INFO] >>>> using GPU (CUDA)
2025-05-14 11:08:42,640 [INFO] >> cyto3 << model set to be used
2025-05-14 11:08:42,771 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-05-14 11:08:43,012 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-05-14 11:08:43,017 [INFO] channels set to [[0, 0]]
2025-05-14 11:08:43,017 [INFO] ~~~ FINDING MASKS ~~~
2025-05-14 11:08:43,019 [INFO] multi-stack tiff read in as having 32 planes 1 channels
2025-05-14 11:08:51,129 [INFO] 100%|##########| 32/32 [00:06<00:00,  4.86it/s]
2025-05-14 11:08:52,197 [INFO] network run in 7.65s
2025-05-14 11:09:

100%|█████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 32.23it/s]


2025-05-14 11:09:20,433 [INFO] masks created in 28.24s
2025-05-14 11:09:31,188 [INFO] >>>> TOTAL TIME 48.17 sec


In [16]:
viewer = napari.view_image(all_img[1], name='img', scale = (4.55,0.3,0.3))
viewer.add_image(masks,name='masks', scale = (4.55,0.3,0.3))

<Image layer 'masks' at 0x1d5d97b1e40>